In [1]:
import pandas as pd
from Bio import SeqIO
import numpy as np
import pandas as pd
import requests

In [ ]:
fasta_file = "/home/musong/python/dataset/swissprot"
# Create an empty dictionary to store sequences
sequences = {}

# Open and parse the FASTA file
with open(fasta_file, "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        # Extract the identifier and remove everything after the dot
        identifier = record.id.split('.')[0]
        sequences[identifier] = str(record.seq)

In [2]:
# add uniprot sequence to the query result
# the query results from web get the sequence from swissprot
def add_sequence(df_path):
    total_missing = 0
    missing_id = []
    df = pd.read_csv(df_path)
    df_sequence = []
    UniProt_id = df['UniProt']
    for target_id in UniProt_id:
        if target_id in sequences:
            df_sequence.append(sequences[target_id])
        else:
            # using grep to find the missing one
            df_sequence.append(-1)
            total_missing = total_missing + 1
            missing_id.append(target_id)
            print(f' {target_id}  is missing')
    print(f'total missing is {total_missing}')
    with open('data/raw_data/missing_id.txt', 'w') as f:
        for _ in missing_id:
            f.write(_)
            f.write('\n')
    return df_sequence

In [50]:
def first_merge():
    df_path = '/home/musong/python/dataset/query results.csv'
    df = pd.read_csv(df_path)
    res = add_sequence(df_path)
    df['sequence'] = res
    df_from_missing_grep = pd.read_csv('/home/musong/python/stu/data/raw_data/result.csv')
    merged_df = pd.merge(df, df_from_missing_grep, left_on='UniProt', right_on='ID', how='left')

    # Drop the 'ID' column (if needed)
    merged_df.drop(columns=['ID'], inplace=True)
    # Creating a new 'sequence' column based on 'sequence_x' and 'sequence_y'
    def combine_sequences(row):
        if row['sequence_x'] == '-1' and row['sequence_y'] == '-1':
            return str('-1')
        elif row['sequence_y'] != '-1' and pd.notna(row['sequence_y']):
            return str(row['sequence_y'])
        else:
            return str(row['sequence_x'])

    merged_df['sequence'] = merged_df.apply(combine_sequences, axis=1)
    # Dropping 'sequence_x' and 'sequence_y' columns
    merged_df.drop(columns=['sequence_x', 'sequence_y'], inplace=True)
    merged_df.to_csv('/home/musong/python/dataset/first_merge.csv', index=False)
first_merge()

 Q99653  is missing
 O95406  is missing
 A0A1B0GTU2  is missing
 P61073  is missing
 Q9UBW8  is missing
 P61604  is missing
 Q8TBE1  is missing
 Q9P003  is missing
 Q92600  is missing
 P0DPH9  is missing
 P99999  is missing
 Q9UBT7  is missing
 Q9BZC1  is missing
 Q8N1G2  is missing
 Q99439  is missing
 Q6PUV4  is missing
 A0A140G945  is missing
 Q9P013  is missing
 Q9H2H0  is missing
 Q9UIV1  is missing
 Q9NPA8  is missing
 P61579  is missing
 Q3ZM63  is missing
 Q9NRF9  is missing
 Q9Y281  is missing
 Q9UKR5  is missing
 Q9UPC5  is missing
 P84090  is missing
 P0CJ86  is missing
 P0CJ88  is missing
 Q9H4M9  is missing
 Q8N9N8  is missing
 P0CL80  is missing
 P0CL82  is missing
 Q9H0R8  is missing
 P63167  is missing
 Q96JC9  is missing
 Q13838  is missing
 O96015  is missing
 Q96FJ2  is missing
 P63172  is missing
 P0CJ89  is missing
 P0CJ90  is missing
 P0DMU4  is missing
 P0DMU5  is missing
 A0A1B0GV22  is missing
 A6NE21  is missing
 Q9H0Q0  is missing
 Q9NUQ9  is missing
 P0DP71 

In [89]:
def get_seq_from_url(uni_id):
    base_url = "https://rest.uniprot.org/uniprotkb/"
    url = f"{base_url}{uni_id}.fasta"
    response = requests.get(url)
    sequence = '-1'
    if response.status_code == 200:
        # Assuming the FASTA format contains the sequence after the first line
        sequence = "".join(response.text.strip().split("\n")[1:])
    else:
        print(f'{uni_id} is missing')
    return sequence

def second_merge():
    df = pd.read_csv('/home/musong/python/dataset/first_merge.csv')

    # Iterate through the DataFrame
    for index, row in df.iterrows():
        sequence_value = row['sequence']
        sequence_id = row['UniProt']
        if sequence_value == '-1':
            sequence = get_seq_from_url(sequence_id)
            df.at[index, 'sequence'] = sequence
            print(f'{sequence_id} is merged')
    df.to_csv('/home/musong/python/dataset/second_merge.csv', index=False, index_label=False)
    return df

In [90]:
second_merge()

A0A140G945 is merged
A6NE21 is merged
A6NEW6 is merged
Q8NEP4 is merged
O15320 is merged
Q6ZRR9 is merged
Q6ZMK1 is merged
P59894 is merged
Q6ZP98 is merged
P0DPR3 is merged
P0DOY5 is merged
Q9NPA5 is merged
P35544 is merged
P0DPI4 is merged
P01858 is merged
Q5VWM5 is merged
Q6ZW33 is merged
Q9UPP5 is merged
B1AJZ1 is merged
P30491 is merged
P30499 is merged
P10316 is merged
Q29718 is merged
P30459 is merged
Q31610 is merged
Q29960 is merged
P30450 is merged
P05534 is merged
P30461 is merged
P03989 is merged
P30483 is merged
P30485 is merged
Q95365 is merged
P01891 is merged
Q29836 is merged
Q95604 is merged
Q9TQE0 is merged
P30685 is merged
P13760 is merged
P13746 is merged
P18463 is merged
Q29974 is merged
Q09160 is merged
Q29963 is merged
P30456 is merged
P30481 is merged
P30486 is merged
Q31612 is merged
P30498 is merged
P30504 is merged
P30453 is merged
P30475 is merged
P18464 is merged
P30490 is merged
P30501 is merged
P01912 is merged
P30508 is merged
Q07000 is merged
P30462 is 

,id,UniProt,Symbol,Name,Target Development Level,IDG Family,Novelty,sequence
0,1,P32929,CTH,Cystathionine gamma-lyase,Tchem,Enzyme,0.001716,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...
1,2,A4D0Y5,C7orf77,Uncharacterized protein C7orf77,Tdark,Other,NaN,MGAERVCTKAPEITQDEAEIYSLTNMEGNIGIKGCEFKSWLFKFYQ...
2,3,Q49A92,C8orf34,Uncharacterized protein C8orf34,Tbio,Other,0.259223,MSSPLASELSELAALRPGFRLSAPHARVAPRAATHARGRGRASHAG...
3,4,Q9UFW8,CGGBP1,CGG triplet repeat-binding protein 1,Tbio,Other,0.108803,MERFVVTAPPARNRSKTALYVTPLDRVTEFGGELHEDGGKLFCTSC...
4,5,Q96K31,C8orf76,Uncharacterized protein C8orf76,Tdark,Other,1.387823,MDSGCWLFGGEFEDSVFEERPERRSGPPASYCAKLCEPQWFYEETE...
...,...,...,...,...,...,...,...,...
20407,20408,P49286,MTNR1B,Melatonin receptor type 1B,Tclin,GPCR,0.006463,MSENGSFANCCEAGGWAVRPGWSGAGSARPSRTPRPPWVAPALSAV...
20408,20409,P84157,MXRA7,Matrix-remodeling-associated protein 7,Tdark,Other,0.208505,MEAPAELLAALPALATALALLLAWLLVRRGAAASPEPARAPPEPAP...
20409,20410,Q9NP71,MLXIPL,Carbohydrate-responsive element-binding protein,Tbio,Transcription Factor,0.003574,MAGALAGLAAGLQVPRVAPSPDSDSDTDSEDPSLRRSAGGLLRSQV...
20410,20411,Q9BWT6,MND1,Meiotic nuclear division protein 1 homolog,Tbio,Other,0.022062,MSKKKGLSAEEKRTRMMEIFSETKDVFQLKDLEKIAPKEKGITAMS...


In [8]:
def third_merge():
    df = pd.read_csv('/home/musong/python/dataset/second_merge.csv')
    nan_index = df[pd.isna(df['sequence'])]
    nan_id = [1449, 1674, 2122, 9827, 15929, 16601, 17587, 18166]
    nan_uniprot = ['A6NE21',
                    'A6NEW6',
                    'Q6ZMK1',
                    'Q5VWM5',
                    'P04745',
                    'Q9Y4X1',
                    'Q13065',
                    'Q13068']
    nan_sequence = ['MMARRDPKSWAKRLVRAQTLQKQRRAPVGPRAPPPDEEDPRLKCKNCGAFGHTARSTRCPMKCWKAALVPATLGKKEGKENLKPWKPRVEANPGPLNKDKGEKEERPRQQDPQRKALLHMFSGKPPEKPLPNGKGSTESSDHLRVASGPMPVHTTSKRPRVDPVLADRSAAEMSGRGSVLASLSPLRKASLSSSSSLGPKERQTGAAADMPQPAVRHQGREPLLVVKPTHSSPEGGCREVPQAASKTHGLLQAARPQAQDKRPAVTSQPCPPAATHSLGLGSNLSFGPGAKRPAQAPIQACLKFPKKPRLGPFQIPESAIQGGELGAPGNLQPPPAATELGPSTSPQMGRRTPAQVPSVDWQPPHSTPCLPTAQACTMSHHSAASHDGAQPLRVLFRRLENGRWSSSLLAAPSFHSPEKPGAFLAQSPHVSEKSEAPCVRVPPSVLYEDLQVSSSSEDSDSDLE',
                    'MMARRDPKSWAKRLVRAQTLQKQRRAPVGPRAPPPDEEDPRLKCKNCGAFGHTARSTRCPMKCWKAALVPATLGKKEGKENLKPWKPRVEANPGPLNKDKGEKEERPRQQDPQRKALLHMFSGKPPEKPLPNGKGSTESSDHLRVASGPMPVHTTSKRPRVDPVLADRSAAEMSGRGSVLASLSPLRKASLSSSSSLGPKERQTGAAADIPQPAVRHQGREPLLVVKPTHSRPEGGCREVPQAASKTHGLLQAARPQAQDKRPAVTSQPCPPAATHSLGLGSNLSFGPGAKRPAQAPIQACLNFPKKPRLGPFQIPESAIQGGELGAPENLQPPPAATELGPSTSPQMGRRTPAQVPSVDRQPPHSTPCLPTAQACTMSHHSAASHDGAQPLRVLFRRLENGRWSSSLLAAPSFHSPEKPGAFLAQSPHVSEKSEAPCVRVPPSVLYEDLQVSSSSEDSDSDLE',
                    'MAPKPGAEWSTALSHLVLGVVSLHAAVSTAEASRGAAAGFLLQVLAATTTLAPGLSTHEDCLAGAWVATVIGLPLLAFDFHWCTNGHLMCAGCFIHLLADARLKEEQATCPNCRCEISKSLCCRNLAVEKAVSELPSECGFCLRQFPRSLLERHQKEECQDRVTQCKYKRIGCPWHGPFHELTVHEAACAHPTKTGSELMEILDGMDQSHRKEMQLYNSIFSLLSFEKIGYTEVQFRPYRTDDFITRLYYETPRFTVLNQTWVLKARVNDSERNPNLSCKRTLSFQLLLKSKVTAPLECSFLLLKGPYDDVRISPVIYHFVFTNESNETDYVPLPIIDSVECNKLLAAKNINLRLFLFQIQK',
                    'MKMSIRTPPRLLELAGRSLLRDQALAMSTLEELPTELFPPLFMEAFSRRRCEALKLMVQAWPFRRLPLRPLIKMPCLEAFQAVLDGLDALLTQGVRPRRWKLQVLDLQDVCENFWMVWSEAMAHGCFLNAKRNKKPVQDCPRMRGRQPLTVFVELWLKNRTLDEYLTYLLLWVKQRKDLLHLCCKKLKILGMPFRNIRSILKMVNLDCIQEVEVNCKWVLPILTQFTPYLGHMRNLQKLVLSHMDVSRYVSPEQKKEIVTQFTTQFLKLRCLQKLYMNSVSFLEGHLDQLLSCLKTSLKVLTITNCVLLESDLKHLSQCPSISQLKTLDLSGIRLTNYSLVPLQILLEKVAATLEYLDLDDCGIIDSQVNAILPALSRCFELNTFSFCGNPICMATLENLLSHTIILKNLCVELYPAPRESYGADGTLCWSRFAQIRAELMNRVRDLRHPKRILFCTDYCPDCGNRSFYDLEADQYCC',
                    'MKLFWLLFTIGFCWAQYSSNTQQGRTSIVHLFEWRWVDIALECERYLAPKGFGGVQVSPPNENVAIHNPFRPWWERYQPVSYKLCTRSGNEDEFRNMVTRCNNVGVRIYVDAVINHMCGNAVSAGTSSTCGSYFNPGSRDFPAVPYSGWDFNDGKCKTGSGDIENYNDATQVRDCRLSGLLDLALGKDYVRSKIAEYMNHLIDIGVAGFRIDASKHMWPGDIKAILDKLHNLNSNWFPEGSKPFIYQEVIDLGGEPIKSSDYFGNGRVTEFKYGAKLGTVIRKWNGEKMSYLKNWGEGWGFMPSDRALVFVDNHDNQRGHGAGGASILTFWDARLYKMAVGFMLAHPYGFTRVMSSYRWPRYFENGKDVNDWVGPPNDNGVTKEVTINPDTTCGNDWVCEHRWRQIRNMVNFRNVVDGQPFTNWYDNGSNQVAFGRGNRGFIVFNNDDWTFSLTLQTGLPAGTYCDVISGDKINGNCTGIKIYVSDDGKAHFSISNSAEDPFIAIHAESKL',
                    'MLNNLLLFSLQISLIGTTLGGNVLIWPMEGSHWLNVKIIIDELIKKEHNVTVLVASGALFITPTSNPSLTFEIYRVPFGKERIEGVIKDFVLTWLENRPSPSTIWRFYQEMAKVIKDFHMVSQEICDGVLKNQQLMAKLKKSKFEVLVSDPVFPCGDIVALKLGIPFMYSLRFSPASTVEKHCGKVPYPPSYVPAVLSELTDQMSFTDRIRNFISYHLQDYMFETLWKSWDSYYSKALGRPTTLCETMGKAEIWLIRTYWDFEFPRPYLPNFEFVGGLHCKPAKPLPKEMEEFIQSSGKNGVVVFSLGSMVKNLTEEKANLIASALAQIPQKVLWRYKGKKPATLGNNTQLFDWIPQNDLLGHPKTKAFITHGGTNGIYEAIYHGVPMVGVPMFADQPDNIAHMKAKGAAVEVNLNTMTSVDLLSALRTVINEPSYKENAMRLSRIHHDQPVKPLDRAVFWIEFVMRHKGAKHLRVAAHDLTWFQYHSLDVIGFLLVCVTTAIFLVIQCCLFSCQKFGKIGKKKKRE',
                    'MSWRGRSTYYWPRPRRYVQPPEMIGPMRPEQFSDEVEPATPEEGEPATQRQDPAAAQEGEDEGASAGQGPKPEADSQEQGHPQTGCECEDGPDGQEMDPPNPEEVKTPEEEMRSHYVAQTGILWLLMNNCFLNLSPRKP',
                    'MSWRGRSTYYWPRPRRYVQPPEMIGPMRPEQFSDEVEPATPEEGEPATQRQDPAAAQEGEDEGASAGQGPKPEADSQEQGHPQTGCECEDGPDGQEMDPPNPEEVKTPEEGEGQSQC']
    for i in range(len(nan_id)):
        df.at[nan_id[i], 'sequence'] = nan_sequence[i]
    
    df.to_csv('/home/musong/python/dataset/third_merge.csv', index=False, index_label=False)
    return df

third_merge()

,id,UniProt,Symbol,Name,Target Development Level,IDG Family,Novelty,sequence
0,1,P32929,CTH,Cystathionine gamma-lyase,Tchem,Enzyme,0.001716,MQEKDASSQGFLPHFQHFATQAIHVGQDPEQWTSRAVVPPISLSTT...
1,2,A4D0Y5,C7orf77,Uncharacterized protein C7orf77,Tdark,Other,NaN,MGAERVCTKAPEITQDEAEIYSLTNMEGNIGIKGCEFKSWLFKFYQ...
2,3,Q49A92,C8orf34,Uncharacterized protein C8orf34,Tbio,Other,0.259223,MSSPLASELSELAALRPGFRLSAPHARVAPRAATHARGRGRASHAG...
3,4,Q9UFW8,CGGBP1,CGG triplet repeat-binding protein 1,Tbio,Other,0.108803,MERFVVTAPPARNRSKTALYVTPLDRVTEFGGELHEDGGKLFCTSC...
4,5,Q96K31,C8orf76,Uncharacterized protein C8orf76,Tdark,Other,1.387823,MDSGCWLFGGEFEDSVFEERPERRSGPPASYCAKLCEPQWFYEETE...
...,...,...,...,...,...,...,...,...
20407,20408,P49286,MTNR1B,Melatonin receptor type 1B,Tclin,GPCR,0.006463,MSENGSFANCCEAGGWAVRPGWSGAGSARPSRTPRPPWVAPALSAV...
20408,20409,P84157,MXRA7,Matrix-remodeling-associated protein 7,Tdark,Other,0.208505,MEAPAELLAALPALATALALLLAWLLVRRGAAASPEPARAPPEPAP...
20409,20410,Q9NP71,MLXIPL,Carbohydrate-responsive element-binding protein,Tbio,Transcription Factor,0.003574,MAGALAGLAAGLQVPRVAPSPDSDSDTDSEDPSLRRSAGGLLRSQV...
20410,20411,Q9BWT6,MND1,Meiotic nuclear division protein 1 homolog,Tbio,Other,0.022062,MSKKKGLSAEEKRTRMMEIFSETKDVFQLKDLEKIAPKEKGITAMS...
